# Scratch

In [1]:
EMPFILE = "/Users/adarshrp/Projects/flare/data/emp.csv"
emp = spark.read.option("header", True).option("inferSchema", True).csv(EMPFILE)
emp.printSchema()
emp.createOrReplaceTempView("emp")


root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- dept_id: integer (nullable = true)



In [22]:
spark.sql("""
select name from EMP
where age >= (select 1 from EMP)
""").show(10)

Py4JJavaError: An error occurred while calling o101.showString.
: java.lang.RuntimeException: more than one row returned by a subquery used as an expression:
Subquery scalar-subquery#99, [id=#239]
+- *(1) Project [1 AS 1#100]
   +- FileScan csv [] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/Users/adarshrp/Projects/flare/data/emp.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<>

	at scala.sys.package$.error(package.scala:30)
	at org.apache.spark.sql.execution.ScalarSubquery.updateResult(subquery.scala:85)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$waitForSubqueries$1(SparkPlan.scala:248)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$waitForSubqueries$1$adapted(SparkPlan.scala:247)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apa

In [ ]:


spark.sql("""
WITH CTE1 AS (select '--cte1', age, '--cte1' from emp),
 CTE2 AS (select '--cte2', age, '--cte2' from emp)
SELECT * FROM CTE1, (select '2nd', dept_id, name from EMP) CTE1
WHERE name in (select name from emp CTE1 where CTE1.age >= 0) 
and age in (select CTE2.age from EMP CTE2 where CTE1.age = CTE2.age)
""").show(100)

In [35]:
parquetFile = spark.read.parquet("/tmp/file.parquet")
parquetFile.createOrReplaceTempView("parquetFile")

In [36]:
parquetFile

DataFrame[column_1: bigint, column_2: string, column_3: string]

In [37]:
parquetFile.columns

['column_1', 'column_2', 'column_3']

In [38]:
spark.sql("SELECT * FROM parquetFile").show(100)

+--------+-----------+--------------------+
|column_1|   column_2|            column_3|
+--------+-----------+--------------------+
|       1|    AMERICA|hs use ironic, ev...|
|       2|       ASIA|ges. thinly even ...|
|       3|     EUROPE|ly final courts c...|
|       4|MIDDLE EAST|uickly special ac...|
+--------+-----------+--------------------+



In [41]:
parquetFile.rdd.getNumPartitions()

1

In [42]:
parquetFile.write.format("parquet").partitionBy("column_2").mode("overwrite").save("/tmp/pqptns.parquet")